Imports

In [ ]:
#setting system path
import os, sys
# Get the path to the project root (one level up from /jupyter)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Add it to the system path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

#Python libraries
from langgraph.graph import StateGraph


#Import Project Libraries
from src.llm_chain.chains import (_ambiguity_checker_chain,
                                  _clarifiaction_ques_generation_chain,
                                  _query_consolidator_chain,
                                  _llm_with_context_chain )
from src.schema.state_schema import StateSchema

In [ ]:
my_query = "who is avnoor?"

In [ ]:
#initial state
agent_state = StateSchema(original_user_query=my_query)

In [ ]:


graph = StateGraph(StateSchema)

#Create nodes
graph.add_node("ambiguity_checker", _ambiguity_checker_chain)
graph.add_node("clarification_ques_gen", _clarifiaction_ques_generation_chain)
